In [1]:
import re
import math

import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

In [2]:
def calc_grs(user_calls,odds_ratios):
    grs = 0
    user_dosages = {}
    for rsid in odds_ratios:
        call = user_calls[rsid]
        odds = odds_ratios[rsid]['odds']
        effect_allele = odds_ratios[rsid]['eff']
        maf = odds_ratios[rsid]['maf']
        
        if call == '00':
            # Missing genotype
            dosage = 2*maf
        else:
            #print(effect_allele)
            dosage = call.count(effect_allele)
            
        grs  = grs + (math.log(odds)*(dosage))
        user_dosages[rsid] = dosage
    
    return grs, user_dosages

In [3]:
with open('./data/sample_disease_info.txt','r') as inp:
    dat = inp.readlines()

disease_status = {'PD':[],'ET':[]}
for line in dat[1:]:
    line = line.strip().split('\t')
    user_id = re.sub('sc0+','',line[0])
    user_id = user_id[:-1]
    disease_status[line[1]].append(user_id)


## File Paths

In [4]:
uk_biobank_data = './data/genetic_data/parkinsons_rsid_selected.ped'
uk_biobank_snp_map = './data/genetic_data/parkinsons_rsid_selected.map'

In [5]:
snp_mapping = {}
with open(uk_biobank_snp_map, 'r') as inp:
    dat = inp.readlines()
    snp_index = 1
    for line in dat:
        line = line.split('\t')
        rsid = line[1]
        snp_mapping[snp_index] = rsid
        snp_index += 1

print(snp_mapping)

{1: 'rs35749011', 2: 'rs823118', 3: 'rs16857578', 4: 'rs6430538', 5: 'rs1474055', 6: 'rs10513789', 7: 'rs34311866', 8: 'rs11724635', 9: 'rs6812193', 10: 'rs9275503', 11: 'rs199355', 12: 'rs117896735', 13: 'rs329648', 14: 'rs76904798', 15: 'rs11060180', 16: 'rs11158026', 17: 'rs11627207', 18: 'rs2414739', 19: 'rs14235', 20: 'rs5026246', 21: 'rs4130047', 22: 'rs8118008'}


In [6]:
# Data stored in dictionary as follows
# {'user1':{'snp1':'AA','snp2':'AB', ...}, 'user2':{'snp1':'AB','snp2':'AB, ...}, ...}

snp_info = {}

In [7]:
with open(uk_biobank_data, 'r') as inp:
    dat = inp.readlines()
    
for line in dat:
    line = line.strip().split(' ')
    user_id = line[1]
    genotypes = line[6:]
    
    snp_info[user_id] = {}
    
    n = 1
    while n<=len(snp_mapping):
        call = genotypes[(n-1)*2]+genotypes[(n-1)*2+1]
        rsid = snp_mapping[n]
        snp_info[user_id][rsid] = call
        n += 1     

## SNP Odds Ratios

In [8]:
odds_ratio_file = './data/genetic_data/selected_snp_or.tsv'

In [9]:
odds_ratios = {}

with open(odds_ratio_file,'r') as inp:
    dat = inp.readlines()

for line in dat[1:]:
    line = line.strip().split('\t')
    rsid = line[0]
    effect = line[1]
    alternate = line[2]
    minor_allele_frequency = float(line[3])
    odds_effect = float(line[4])    
    
    odds_ratios[rsid] = {}
    odds_ratios[rsid]['eff'] = effect
    odds_ratios[rsid]['alt'] = alternate
    odds_ratios[rsid]['odds'] = odds_effect
    odds_ratios[rsid]['maf'] = minor_allele_frequency   



odds_ratios

{'rs10513789': {'alt': 'T', 'eff': 'G', 'maf': 0.193, 'odds': 0.842},
 'rs11060180': {'alt': 'G', 'eff': 'A', 'maf': 0.558, 'odds': 1.105},
 'rs11158026': {'alt': 'C', 'eff': 'T', 'maf': 0.335, 'odds': 0.904},
 'rs11627207': {'alt': 'G', 'eff': 'T', 'maf': 0.468, 'odds': 0.897},
 'rs11724635': {'alt': 'C', 'eff': 'A', 'maf': 0.553, 'odds': 1.126},
 'rs117896735': {'alt': 'G', 'eff': 'A', 'maf': 0.014, 'odds': 1.624},
 'rs14235': {'alt': 'G', 'eff': 'A', 'maf': 0.381, 'odds': 1.103},
 'rs1474055': {'alt': 'C', 'eff': 'T', 'maf': 0.128, 'odds': 1.214},
 'rs16857578': {'alt': 'C', 'eff': 'T', 'maf': 0.14, 'odds': 1.131},
 'rs199355': {'alt': 'G', 'eff': 'A', 'maf': 0.59, 'odds': 1.11},
 'rs2414739': {'alt': 'G', 'eff': 'A', 'maf': 0.734, 'odds': 1.113},
 'rs329648': {'alt': 'C', 'eff': 'T', 'maf': 0.354, 'odds': 1.105},
 'rs34311866': {'alt': 'C', 'eff': 'T', 'maf': 0.809, 'odds': 0.786},
 'rs35749011': {'alt': 'G', 'eff': 'A', 'maf': 0.017, 'odds': 1.824},
 'rs4130047': {'alt': 'T', 'eff

In [10]:
outfile = open('./data/genetic_data/snp_grs.txt','w')
GRS = {}
for user in list(snp_info):
    user_calls = snp_info[user]
    grs, user_dosage = calc_grs(user_calls,odds_ratios)
    GRS[user] = grs
    
    outfile.write(str(user)+'\t'+str(grs)+'\n')
    
outfile.close()

In [11]:
et_grs = []
for i in disease_status['ET']:
    try:
        et_grs.append(GRS[i])
    except KeyError:
        pass

pd_grs = []
for i in disease_status['PD']:
    try:
        pd_grs.append(GRS[i])
    except KeyError:
        pass


In [12]:
stats.ttest_ind(et_grs,pd_grs)

Ttest_indResult(statistic=0.44915688761576866, pvalue=0.6549367211050412)